In [1]:
from IPython.core.magic import Magics, cell_magic, line_magic, magics_class
from pexpect import spawn

TIMEOUT = 60
PROG = "hive"
PROMPT = ["\r\n    > ", "\r\nhive> "]
QUIT = "quit;"


@magics_class
class Magic(Magics):
    def __init__(self, shell):
        super().__init__(shell)
        self.app = spawn(PROG, timeout=60)
        self.app.expect(PROMPT)

    @cell_magic
    def hive(self, line, cell):
        cell_lines = [cell_line.strip() for cell_line in cell.split("\n")]
        cell_lines = [cell_line for cell_line in cell_lines if cell_line != ""]
        for cell_line in cell_lines:
            self.app.sendline(cell_line)
            self.app.expect(PROMPT, timeout=TIMEOUT)
            output = self.app.before.decode()
            output = output.replace("\r\n", "\n")
            output = output.split("\n")
            output = [output_line.strip() for output_line in output]
            for output_line in output:
                if output_line not in cell_lines:
                    print(output_line)
        return None

    @line_magic
    def quit(self, line):
        self.app.sendline(QUIT)


def load_ipython_extension(ip):
    ip.register_magics(Magic(ip))


load_ipython_extension(ip=get_ipython())

In [12]:
!hdfs dfs -copyFromLocal data.tsv /tmp

copyFromLocal: `/tmp/data.tsv': File exists


In [13]:
!hdfs dfs -ls /tmp

Found 3 items
-rw-r--r--   1 root supergroup        677 2023-05-30 16:24 /tmp/data.tsv
drwxrwx---   - root supergroup          0 2023-05-30 16:19 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2023-05-30 16:21 /tmp/hive


In [22]:
%%hive
DROP TABLE IF EXISTS data;

OK
Time taken: 0.031 seconds


In [23]:
%%hive
CREATE TABLE data (
letter STRING,
date_event STRING,
value INT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY  '\t';

OK
Time taken: 0.122 seconds


In [24]:
%%hive
SHOW TABLES;

OK
data
Time taken: 0.031 seconds, Fetched: 1 row(s)


In [25]:
%%hive
LOAD DATA INPATH '/tmp/data.tsv' OVERWRITE INTO TABLE data;
SELECT * FROM data;

Loading data to table default.data
OK
Time taken: 0.889 seconds
OK
B	1999-08-28	14
E	1999-12-06	12
E	1993-07-21	17
C	1991-02-12	13
E	1995-04-25	16
A	1992-08-22	14
B	1999-06-11	12
E	1993-01-27	13
E	1999-09-10	11
E	1990-05-03	16
E	1994-02-14	5
A	1988-04-27	12
A	1990-10-06	10
E	1985-02-12	16
E	1998-09-14	16
B	1994-08-30	17
A	1997-12-15	13
B	1995-08-23	10
B	1998-11-22	13
B	1997-04-09	14
E	1993-12-27	18
E	1999-01-14	15
A	1992-09-19	18
B	1993-03-02	14
B	1999-10-21	13
A	1990-08-31	12
C	1994-01-25	6
E	1990-02-09	18
A	1990-09-26	14
A	1993-05-08	16
B	1995-09-06	14
E	1991-02-18	14
A	1993-01-11	14
A	1990-07-22	18
C	1994-09-09	15
C	1994-07-27	7
D	1990-10-10	15
A	1990-09-05	11
B	1991-10-01	15
A	1994-10-25	13
Time taken: 1.277 seconds, Fetched: 40 row(s)


In [28]:
%%hive
SELECT letter letra, COUNT(*) cantidad
FROM data
GROUP BY letter;

Query ID = root_20230530163823_7067e4da-372e-434c-8416-ee5d38924b8c
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
set mapreduce.job.reduces=<number>
Starting Job = job_1685463534098_0001, Tracking URL = http://a0b990b2c7b2:8088/proxy/application_1685463534098_0001/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1685463534098_0001
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2023-05-30 16:38:38,194 Stage-1 map = 0%,  reduce = 0%
2023-05-30 16:38:44,720 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 3.02 sec
2023-05-30 16:38:52,109 Stage-1 map = 100%,  reduce = 100%, Cumulative CPU 6.57 sec
MapReduce Total cumulative CPU time: 6 seconds 570